In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from imutils import paths
from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import joblib
import cv2
from sklearn.utils.extmath import softmax
from keras import models
from keras import layers
from keras import optimizers
import tensorflow as tf
from tensorflow import keras

In [6]:
!unzip /content/drive/MyDrive/Test/Test.zip


Archive:  /content/drive/MyDrive/Test/Test.zip
   creating: Test/
   creating: Test/astilbe/
  inflating: Test/astilbe/14401005141_658fe992f7_c.jpg  
  inflating: Test/astilbe/14413059366_7e8f031b74_c.jpg  
  inflating: Test/astilbe/14525825511_9540550d85_c.jpg  
  inflating: Test/astilbe/14529200855_da4f82715e_c.jpg  
  inflating: Test/astilbe/15349874996_d875615d13_c.jpg  
  inflating: Test/astilbe/15369727941_8047abb811_c.jpg  
  inflating: Test/astilbe/18931747274_13941bf5b8_c.jpg  
  inflating: Test/astilbe/18990523683_8c75955d48_c.jpg  
  inflating: Test/astilbe/20972594356_16b8c4c11c_c.jpg  
  inflating: Test/astilbe/21090361301_95860bd711_c.jpg  
  inflating: Test/astilbe/2702350884_5550de9287_c.jpg  
  inflating: Test/astilbe/2702361902_9f0e53d501_c.jpg  
  inflating: Test/astilbe/27701562400_e3e76c062f_c.jpg  
  inflating: Test/astilbe/27717834455_6535ed5e11_c.jpg  
  inflating: Test/astilbe/28457229503_b777a5d276_c.jpg  
  inflating: Test/astilbe/28546323030_b4b17ae257_c.jpg

In [8]:
# Lấy các đường dẫn đến ảnh.
image_path_test = list(paths.list_images('Test/'))
print(image_path_test)

['Test/calendula/2365071287_dc34bd9223_c.jpg', 'Test/calendula/2718835573_121cbd3b7a_c.jpg', 'Test/calendula/3274311728_ed0f68f840_c.jpg', 'Test/calendula/29662533555_324555dc3e_c.jpg', 'Test/calendula/1491917074_db7ca4d627_c.jpg', 'Test/calendula/19920472299_56cfd73974_c.jpg', 'Test/calendula/424215544_9f6c9fb2bb_c.jpg', 'Test/calendula/3379055865_09bf4fd278_c.jpg', 'Test/calendula/3281499647_1e9ea96a2c_c.jpg', 'Test/calendula/2805136718_d6cc687d27_c.jpg', 'Test/calendula/2978855063_3dd71bf294_c.jpg', 'Test/calendula/2279523308_4a697f4c83_c.jpg', 'Test/calendula/28346689321_7799ab1b69_c.jpg', 'Test/calendula/2044029910_f519ea07c3_c.jpg', 'Test/calendula/2563781741_022a70b2b9_c.jpg', 'Test/calendula/49237512801_75e16f9a08_c.jpg', 'Test/calendula/427088546_1b5b753785_c.jpg', 'Test/calendula/3080046092_96fff00c90_c.jpg', 'Test/calendula/2799112321_fae839d391_c.jpg', 'Test/calendula/3320051061_c70ef707db_c.jpg', 'Test/calendula/2391132785_bfff2ac869_c.jpg', 'Test/calendula/2279523280_27bb

In [ ]:
model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels_test = [p.split(os.path.sep)[-2] for p in image_path_test]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels_test = le.fit_transform(labels_test)

lb = LabelBinarizer()
labels_test= lb.fit_transform(labels_test)

[[0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 ...
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]]


In [ ]:
list_image_test = []
for (j, imagePath) in enumerate(image_path_test):
    image_test = load_img(imagePath, target_size=(224, 224))
    image_test = img_to_array(image_test)
    
    image_test = np.expand_dims(image_test, 0)
    
    image_test = imagenet_utils.preprocess_input(image_test)
    
    list_image_test.append(image_test)
    
list_image_test = np.vstack(list_image_test)

In [ ]:
features_test = model.predict(list_image_test)

# Giống bước flatten trong CNN, chuyển từ tensor 3 chiều sau ConvNet sang vector 1 chiều
features_test = features_test.reshape((features_test.shape[0], 512*7*7))

In [ ]:
X_test= features_test
y_test = labels_test

In [ ]:
new_model = tf.keras.models.load_model('/content/my_model1.h5')

new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 150534    
                                                                 
Total params: 150,534
Trainable params: 150,534
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred = new_model.predict(X_test,verbose=0)
rounded_labels=np.argmax(y_test, axis=1)
rounded_pred=np.argmax(y_pred, axis=1)
print(confusion_matrix(rounded_labels,rounded_pred))
print(classification_report(rounded_labels,rounded_pred))

[[60  2  0  2  0  0]
 [ 3 83  0  4  5  1]
 [ 0  0 86  3  0  0]
 [ 0  1  4 82  6  1]
 [ 5  5  1  5 92  0]
 [ 0  0  0  0  0 90]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91        64
           1       0.91      0.86      0.89        96
           2       0.95      0.97      0.96        89
           3       0.85      0.87      0.86        94
           4       0.89      0.85      0.87       108
           5       0.98      1.00      0.99        90

    accuracy                           0.91       541
   macro avg       0.91      0.92      0.91       541
weighted avg       0.91      0.91      0.91       541



In [ ]:
results={
   0:'astilbe',
   1:'bellflower',
   2:'black-eyed susan',
   3:'calendula',
   4:'california_poppy',
   5:'tulip'
}
pred=new_model.predict(features_test)
predict_label = []
for item in pred:
  predict_label.append(results[np.argmax(item)])
print(predict_label)

['calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'black-eyed susan', 'calendula', 'calendula', 'calendula', 'black-eyed susan', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'black-eyed susan', 'calendula', 'calendula', 'tulip', 'calendula', 'calendula', 'calendula', 'california_poppy', 'calendula', 'california_poppy', 'calendula', 'calendula', 'black-eyed susan', 'calendula', 'calendula', 'calendula', 'calendula', 'california_poppy', 'california_poppy', 'calendula', 'calendula', 'bellflower', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'calendula', 'california_poppy', 'calendula', 'calendula', 'calendula', 'ca

In [ ]:
sort_pre_lable =   sorted(predict_label)
print(sort_pre_lable)